In [17]:
import pandas as pd

# Read the .data file (assuming it's space-separated; adjust delimiter if needed)
data = pd.read_csv('ann-train.data', delimiter=' ', header=None)

# Save the data to a .csv file
data.to_csv('full_dataset.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch

data = pd.read_csv('full_dataset.csv')

data['target'] = data[['query_hypothyroid', 'query_hyperthyroid', 'TSH_measured']].idxmax(axis=1)

# Map target labels to numeric values
data['target'] = data['target'].map({
    'query_hypothyroid': 0,
    'query_hyperthyroid': 1,
    'TSH_measured': 2
})

# Define features and target
features = ['TSH', 'T3', 'TT4', 'T4U', 'FTI']
target = 'target'

X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

train_data = X_train.copy()
train_data[target] = y_train

model = BayesianNetwork()

hc = HillClimbSearch(train_data, scoring_method=K2Score(train_data))
best_model = hc.estimate()

model.fit(train_data, estimator=MaximumLikelihoodEstimator)

y_pred = model.predict(X_test)
y_pred = y_pred[target]

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

if accuracy >= 0.85:
    print("Expected accuracy is achieved.")
else:
    print("Expected accuracy is not achieved.")